In [ ]:
# Upgrade Oracle ADS to pick up latest features and maintain compatibility with Oracle Cloud Infrastructure.

!pip install -U oracle-ads

Oracle Data Science service sample notebook.

Copyright (c) 2022 Oracle, Inc. All rights reserved. Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.

***

# <font color=red>Audi Autonomous Driving Dataset Repository</font>
<p style="margin-left:10%; margin-right:10%;">by <font color="teal">Oracle for Research</font></p>

***

## Overview:

Audi Autonomous Driving Dataset (A2D2) is an open multi-sensor dataset for autonomous driving research. This dataset consists of simultaneously recorded images and 3D point clouds, together with 3D bounding boxes, semantic segmentation, instance segmentation, and data extracted from the automotive bus. The sensor suite consists of six cameras and five LiDAR units, providing 360-degree coverage.

This notebook demonstrates how to download the dataset from Oracle Cloud Infrastructure (OCI) Object Storage and work with the JSON configuration file. It also demonstrates how to process image and LiDAR data, and display them.

Compatible conda pack: [Computer Vision](https://docs.oracle.com/en-us/iaas/data-science/using/conda-com-vision-fam.htm) for CPU on Python 3.7 (version 1.0)

---

## Contents:

- <a href="#intro">Introduction</a>
    - <a href='#data'>Dataset</a>
    - <a href="#open_data">What is Oracle Open Data?</a>
- <a href="#understand">Download and Understand the Dataset</a>
- <a href='#download'>Download</a>
    - <a href='#config'>Understanding the Configuration File
    - <a href='#lidar'> Understanding the LiDAR Data
- <a href="#images"> Processing Images
    - <a href='#load'> Loading Images
    - <a href='#map'> Mapping LiDAR Data onto Images
- <a href="#cleanup">Clean Up</a>
- <a href='#ref'>References</a>
    
---


Datasets are provided as a convenience. Datasets are considered third-party content and are not considered materials under your agreement with Oracle.

You can access the `Audi Autonomous Driving Dataset (A2D2)` dataset license [here](https://aev-autonomous-driving-dataset.s3.eu-central-1.amazonaws.com/LICENSE.txt).

---


In [ ]:
import cv2
import glob
import json
import matplotlib.pylab as pt
import numpy as np
import os
import pprint
import requests
import tarfile

from os.path import join
from shutil import rmtree
from tempfile import mkdtemp

<a id="intro"></a>
# Introduction

<a id='data'></a>
## Dataset

**Data**: A2D2 includes data recorded on highways, country roads, and cities in the south of Germany. The data are recorded under cloudy, rainy, and sunny weather conditions. The dataset provides semantic segmentation labels, instance segmentation labels, and 3D bounding boxes for non-sequential frames. 41,277 images have semantic and instance segmentation labels for 38 categories. All images have corresponding LiDAR point clouds, of which 12,497 are annotated with 3D bounding boxes within the field of view of the front-center camera. The [A2D2: Audi Autonomous Driving Dataset](https://arxiv.org/pdf/2004.06320.pdf) paper provides more details about how the data are collected and structured.

**Directory Structure**: There are multiple tar files for the sections, such as _camera_ and _lidar_ with the released timestamp in 'YYYYMMDDHHMMSS' format. Each tar file contains images, annotated labels, and point clouds.

**Template**: `https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcxvbiyd8fn/b/a2d2/o/<tar file name>`. For example, 
* The data for camera 'frontcenter' with timestamp '20180810150607': https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcxvbiyd8fn/b/a2d2/o/camera_lidar-20180810150607_camera_frontcenter.tar

**Data Availability**: All data is available from the [Audi Autonomous Driving Dataset (A2D2) repository](https://opendata.oraclecloud.com/ords/r/opendata/opendata/details?data_set_id=6), which is part of [Oracle Open Data](https://opendata.oraclecloud.com/ords/r/opendata/opendata/home).

<a id="open_data"></a>
## What is Oracle Open Data?

Oracle Open Data is a free service that curates spatial images, protein sequences, and annotated text files from the world's leading scientific databases. The repository connects researchers, developers, students, and educators with petabytes of open data from trusted resources. You can use Oracle Open Data to view important metadata and sample code for each data set, which simplifies technical complexities and makes it easy for researchers to use.

<a id="understand"></a>
# Download and Understand the Dataset

<a id="download"></a>
## Download
OCI Object Storage enables you to securely store any type of data in its native format. With built-in redundancy, Object Storage is ideal for building modern applications that require scale and flexibility because you can use it to consolidate multiple data sources for analytics, backup, or archive purposes.

The Caltech pedestrian detection benchmark for video data are stored in Object Storage in the SEQ format. You can run the next cell to download the `a2d2-preview.tar` and extract all files.

In [ ]:
a2d2_preview_url = "https://objectstorage.us-ashburn-1.oraclecloud.com/n/idcxvbiyd8fn/b/a2d2/o/a2d2-preview.tar"
print("Downloading A2D2 Preview Dataset...")
response = requests.get(a2d2_preview_url, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|tar")
data_path = mkdtemp()
print(f"Extracting data to {data_path}")
file.extractall(path=data_path)
print("Download completed")

<a id="config"></a>
## Understanding the Configuration File

Each `.tar` file includes a `cams_lidars.json` file that plays an important role as the configuration file in processing the A2D2 dataset. This JSON file contains three main items:

In [ ]:
with open(join(data_path, "cams_lidars.json"), "r") as config:
    config = json.load(config)
config.keys()

Each sensor in the keys `lidars`, `cameras`, and `vehicle` have an associated `view`. A view is a sensor coordinate system, defined by an origin, and an x and y-axis. These are specified in cartesian coordinates (in m) relative to an external coordinate system. Unless otherwise stated the external coordinate system is the car's frame of reference.

In [ ]:
config["vehicle"].keys()

The `vehicle` key contains a `view` object specifying the frame of reference of the car. It also contains an `ego-dimensions` object, which specifies the extension of the vehicle in the frame of reference of the car.

In [ ]:
config["lidars"].keys()

The `lidars` key contains objects specifying the extrinsic calibration parameters for each LiDAR sensor. The car has five LiDAR sensors `front_left`, `front_center`, `front_right`, `rear_right`, and `rear_left`'. Each LiDAR has a `view` in the frame of reference of the car.

In [ ]:
config["cameras"].keys()

The `cameras` key contains camera objects which specify their calibration parameters. The car has six cameras `front_left`, `front_center`, `front_right`, `side_right`, `rear_center`, and `side_left`.

<a id="lidar"></a>
## Understanding the LiDAR Data

This section demonstrates how to read point clouds corresponding to the camera. The `front_center` camera is used as an example in the next cell. The LiDAR data are saved in compressed numpy format and can be read using `np.load`.

In [ ]:
root_path = join(data_path, "camera_lidar_semantic_bboxes")
file_names = sorted(glob.glob(join(root_path, "*/lidar/cam_front_center/*.npz")))

file_name_lidar = file_names[0]  # select the lidar point cloud. You are able to
# change this to view different images.

lidar_front_center = np.load(file_name_lidar)

You can explore the LiDAR data using the LiDAR points within the field of view of the front center camera.

In [ ]:
list(lidar_front_center.keys())

For example, you can access the coordinates of LiDAR points `row` and `col` in image space.

In [ ]:
row = lidar_front_center["row"]
col = lidar_front_center["col"]
(row, col)

<a id="images"></a>
# Processing Images
<a id="load"></a>
## Loading Images

The next cell parses a LiDAR filename, `file_name_lidar`. This information is used to create an image filename for the front, center camera image.

In [ ]:
file_parse = file_name_lidar.split("/")[-1].split(".")[0].split("_")
file_name_image = join(
    root_path,
    f"{file_parse[0][0:8]}_{file_parse[0][8:]}",
    "camera",
    "cam_front_center",
    f"{file_parse[0]}_camera_frontcenter_{file_parse[3]}.png",
)
file_name_image

OpenCV-Python is a library of Python bindings designed to solve computer vision problems. In the next cell, `cv2.imread()` loads an image with the color channel order of the BGR (blue, green, red). The `cv2.cvtColor()` method is used to convert the image to the RGB (red, green, blue) color space and the image is saved.

Then you can visualize the selected image captured by the front center camera.

In [ ]:
image_front_center = cv2.imread(file_name_image)
image_front_center = cv2.cvtColor(image_front_center, cv2.COLOR_BGR2RGB)

# display image from front center camera
pt.fig = pt.figure(figsize=(15, 15))
pt.imshow(image_front_center)
pt.axis("off")
pt.title("front center")
pt.show();

<a id="map"></a>
## Mapping LiDAR Data onto Images

This section demonstrates how to map LiDAR data onto images. The next cell defines the function `hsv_to_rgb()` to convert a hue, saturation, value (HSV) image into RGB format. The function accepts three parameters, `h`, `s`, `v`, which represent hue, saturation, value respectively. The [Wikipedia page](https://en.wikipedia.org/wiki/HSL_and_HSV) details how this function is defined.

In [ ]:
def hsv_to_rgb(h, s, v):
    if s == 0.0:
        return v, v, v

    i = int(h * 6.0)
    f = (h * 6.0) - i
    p = v * (1.0 - s)
    q = v * (1.0 - s * f)
    t = v * (1.0 - s * (1.0 - f))
    i = i % 6

    if i == 0:
        return v, t, p
    if i == 1:
        return q, v, p
    if i == 2:
        return p, v, t
    if i == 3:
        return p, q, v
    if i == 4:
        return t, p, v
    if i == 5:
        return v, p, q

The following cell defines the function `map_lidar_points_onto_image`. It maps LiDAR points onto the image. This function accepts four parameters, where `image_orig` is, for example, `image_front_center`, `lidar` represents the data you loaded in the section of <a href='#lidar'>Understanding the LiDAR Data<a>, `pixel_size` and `pixel_opacity`.

In [ ]:
def map_lidar_points_onto_image(image_orig, lidar, pixel_size=3, pixel_opacity=1):
    image = np.copy(image_orig)

    # get rows and cols
    rows = (lidar["row"] + 0.5).astype(np.int32)
    cols = (lidar["col"] + 0.5).astype(np.int32)

    # lowest distance values to be accounted for in colour code
    MIN_DISTANCE = np.min(lidar["distance"])
    # largest distance values to be accounted for in colour code
    MAX_DISTANCE = np.max(lidar["distance"])

    # get distances
    distances = lidar["distance"]
    # determine point colours from distance
    colours = (distances - MIN_DISTANCE) / (MAX_DISTANCE - MIN_DISTANCE)
    colours = np.asarray(
        [np.asarray(hsv_to_rgb(0.75 * c, np.sqrt(pixel_opacity), 1.0)) for c in colours]
    )
    pixel_rowoffs = np.indices([pixel_size, pixel_size])[0] - pixel_size // 2
    pixel_coloffs = np.indices([pixel_size, pixel_size])[1] - pixel_size // 2
    canvas_rows = image.shape[0]
    canvas_cols = image.shape[1]
    for i in range(len(rows)):
        pixel_rows = np.clip(rows[i] + pixel_rowoffs, 0, canvas_rows - 1)
        pixel_cols = np.clip(cols[i] + pixel_coloffs, 0, canvas_cols - 1)
        image[pixel_rows, pixel_cols, :] = (1.0 - pixel_opacity) * np.multiply(
            image[pixel_rows, pixel_cols, :], colours[i]
        ) + pixel_opacity * 255 * colours[i]
    return image.astype(np.uint8)

Now, you can visualize the mapping of the LiDAR point clouds onto the front center image.

In [ ]:
image = map_lidar_points_onto_image(image_front_center, lidar_front_center)
pt.fig = pt.figure(figsize=(15, 15))
pt.imshow(image)
pt.axis("off")
pt.show();

<a id="cleanup"></a>
# Clean Up

This notebook downloaded several gigabytes of data. This section will delete those files.

In [ ]:
rmtree(data_path)

<a id='ref'></a>
# References
- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)
- [Understanding Conda Environments](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)
- [Use Resource Manager to Configure Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/orm-configure-tenancy.htm)
- [A2D2 at Oracle Open Data](https://opendata.oraclecloud.com/ords/r/opendata/opendata/details?data_set_id=6)
- [A2D2: Audi Autonomous Driving Dataset](https://arxiv.org/pdf/2004.06320.pdf)
- [A2D2 Tutorial](https://www.a2d2.audi/a2d2/en/tutorial.html)
- [Convert BGR and RGB with Python, OpenCV](https://note.nkmk.me/en/python-opencv-bgr-rgb-cvtcolor/)